# 1. Inspect the website's source code

### HTML Refresher
This part is based on chapter 11 of *Automate the Boring Stuff with Python* by Al Sweigart

HTML files are plain text files containing *tags*, which are words enclosed in angle brackets. Tags tell the browser how to format the web page. A starting tag and closing tag can enclose some text to form an element. The text (or inner HTML) is the content between the starting and closing tags.

There are many different tags in HTML. Some of these tags have extra properties in the form of attributes within the angle brackets. For example, the `<a>` tag encloses text that should be a link.

Some elements have an `id` attribute that is used to uniquely identify the element in the page. You will often instruct your programs to seek out an element by its id attribute, so figuring out an element’s id attribute using the browser’s developer tools is a common task in writing web scraping programs.

### View a page's HTML source

In Firefox:
To view a page's sources right click on it and choose **View page source** which opens a new tab with the HTML sources.
<img src="resources/img/view_page_source.png" width="500"> 

# 2. Fetching data
Use Python's `requests` library

In [ ]:
import requests

url = 'https://github.com/benesom/redi-da-cph-spring21'
r = requests.get(url)
r.text

# 3. Parse data and extract from it with BeautifulSoup

BeautifulSoup is a module for parsing and extracting information from HTML sources. The module’s name is bs4. In case it is not already installed on your machine:
- install it with 
```bash 
pip install beautifulsoup4

While beautifulsoup4 is the name used for installation, to import BeautifulSoup in your notebook you have to use `import bs4`.

Documentation: https://www.crummy.com/software/BeautifulSoup/

_"Beautiful Soup parses anything you give it, and does the tree traversal stuff for you. You can tell it "Find all the links", or "Find all the links of class externalLink", or "Find all the links whose urls match "foo.com", or "Find the table heading that's got bold text, then give me that text."_

### A Creating a BeautifulSoup Object from our local HTML File

- The `bs4.BeautifulSoup()` function needs to be called with a string containing the HTML file it will parse and returns a `BeautifulSoup` object.

You can load a local HTML file and pass a file object to `bs4.BeautifulSoup()`.

In [ ]:
import bs4

with open('resources/example.html') as f:
    example_html = f.read()
    
soup = bs4.BeautifulSoup(example_html)
print(type(soup))
#print(soup.prettify())

### B Creating a BeautifulSoup object from a website

In [ ]:
import requests

url = 'https://github.com/benesom/redi-da-cph-spring21'

r = requests.get(url)
r.raise_for_status()
soup = bs4.BeautifulSoup(r.text, 'html.parser')

print(soup.prettify()[:1500])

## Finding an Element with the `select()` Method

You can retrieve HTML elements from a `BeautifulSoup` object by calling the `select()` method and passing a string of a CSS selector for the element you are looking for. Selectors are like regular expressions: They specify a pattern to look for, in this case, in HTML pages instead of general text strings.

Common CSS selector patterns include:

  * `soup.select('div')` ... selects all elements named `<div>`
  * `soup.select('#lecturer')`  ... selects the element with an id attribute of author
  * `soup.select('.notice')` ... selects all elements that use a CSS class attribute named notice
  * `soup.select('div span')` ... selects all elements named `<span>` that are within an element named `<div>`
  * `soup.select('div > span')` ... selects all elements named `<span>` that are directly within an element named `<div>`, with no other element in between
  * `soup.select('input[name]')` ... selects all elements named `<input>` that have a name attribute with any value
  * `soup.select('input[type="button"]')` ... selects all elements named `<input>` that have an attribute named type with value button
  
See more in the documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors

In [ ]:
with open('resources/example.html') as f:
    example_html = f.read()

soup = bs4.BeautifulSoup(example_html, 'html.parser')

elems = soup.select('body')

#print(soup.prettify())
print('1: return type of select()',type(elems))
print('2: length of the returned list',len(elems))
print('3: type of elements in the list',type(elems[0]))
print('4: get text from the element',elems[0].getText()[:40])
print('5: string representation of an element: ',str(elems[0]))

#### Extracting a link

In [ ]:
a_elems = soup.select('a')
a_elems[0]['href']

### What is the difference between the `select` and the `find`/`find_all` functions?

You are not the first ones wondering about this... See:
https://stackoverflow.com/questions/38028384/beautifulsoup-is-there-a-difference-between-find-and-select-python-3-x#38033910

# Some Notes

**OBS** Many web pages are not built to support high traffic or they explicitely discourage automatic access. Keep this in mind when writing your scraping tool.

If you are interested in more web scraping and application of crawlers have a look at the `scrapy` module (https://scrapy.org)

In [ ]:
from time import sleep
sleep(3) # script doesn't continue for 3 seconds

# Example Scraping Events from a Page

Let's scrape some apartment prices from boliga.dk


In [2]:
import bs4
import requests

url = "https://www.boliga.dk/salg/resultater?page=1&sort=date-d&salesDateMax=2021&street=&municipality=101"

r = requests.get(url)
r.raise_for_status
soup = bs4.BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   window.dataLayer = window.dataLayer || [];
    window.dataLayer.push({
      originalLocation: document.location.protocol + '//' +
                        document.location.hostname +
                        document.location.pathname +
                        document.location.search
    });
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-PWD5VZT');
  </script>
  <!-- End Google Tag Manager -->
  <meta charset="utf-8"/>
  <meta content="Se udbudspriser og salgspriser på boliger i Danmark baseret på din søgning. Se resultaterne på en liste her." name="description"/>
  <meta content="

In [3]:
rows = soup.find_all('tr', attrs={'class': 'table-row'})
len(rows)

50

In [4]:
result_list = []
for row in rows:
    bolig_dict = {}
    cols = row.find_all('td')
    bolig_dict['address'] = cols[0].text
    bolig_dict['price'] = cols[1].text
    bolig_dict['size'] = cols[6].text
    bolig_dict['rooms'] = cols[5].text
    bolig_dict['built_year'] = cols[7].text
    result_list.append(bolig_dict)

result_list[0]
    

{'address': ' Trelleborggade 18, st. tv, 2150 Nordhavn ',
 'price': ' Købesum 3.250.000',
 'size': ' m²  54 m² ',
 'rooms': ' Værelser  2 ',
 'built_year': ' Byggeår  2016 '}

In [5]:
import pandas as pd

df = pd.DataFrame(result_list)
df.head()

,address,price,size,rooms,built_year
0,"Trelleborggade 18, st. tv, 2150 Nordhavn",Købesum 3.250.000,m² 54 m²,Værelser 2,Byggeår 2016
1,"Trelleborggade 14, 1. tv, 2150 Nordhavn",Købesum 6.495.000,m² 116 m²,Værelser 4,Byggeår 2016
2,"Enghavevej 15, st. th, 1674 København V",Købesum 2.795.000,m² 82 m²,Værelser 3,Byggeår 1895
3,"Eckersbergsgade 6, 2100 København Ø",Købesum 12.000.000,m² 135 m²,Værelser 5,Byggeår 1883
4,"Holtegade 12, 1. mf, 2200 København N",Købesum 1.020.000,m² 65 m²,Værelser 2,Byggeår 1932
